## outline

1. introduction  / problem description
1. basics from graph theory
1. mathematical formulation
1. pyomo implementation
1. example
1. summary or outlook for other solution methods
1. gale shapley alggorithmus: https://core.ac.uk/download/pdf/82241772.pdf, https://www.geeksforgeeks.org/stable-marriage-problem/
reference: http://cgm.cs.mcgill.ca/~avis/courses/251/2012/ktlaerid/LP-bliss.pdf , https://econcs.seas.harvard.edu/files/econcs/files/wang_thesis16.pdf, 
1. compare to SAT, i.e. ORTools: https://planspace.org/20230316-solving_a_matching_problem_with_ortools_cpsat/


## algebraic description

Following [@vate1989linear] the stable marriage problem can be formulated as linear program as follows:

$$
\begin{array}{lll}
\min & \sum x_{ij} & \\
s.t. & \sum_j x_{ij} = 1 & \forall i \\
     & \sum_i x_{ij} = 1 & \forall j \\
     & \sum_{k: w_k <_{m_i} w_j} x_{ik} + \sum_{k:m_k\leq_{w_j} m_i} x_{kj} \leq 1& \forall i,j\\
     & x_{ij} \geq 0 & \forall i,j
\end{array}
$$


In [ ]:
import pyomo.environ as pyo
import pandas as pd
import numpy as np

In [ ]:
set_size = 6

In [ ]:
def gen_source(number):
    """returns list of unique strings representing sources, i.e. 's1'"""
    return ['s' + str(i) for i in range(number)]
def gen_target(number):
    """returns list of unique strings representing targets, i.e. 't1'"""
    return ['t' + str(i) for i in range(number)]
def gen_preferences(number):
    """generate array of size number x number whose elements represent the preference
    rows =, columns = 
    """
    def _helper_gen_vec(number):
        """gnerates random vector of integers from 0 to number without replacement"""
        out = np.random.choice(number, replace = False, size = (1, number)) + 1
        return out.tolist()                        
    output = []
    for i in range(number):
        output = output + _helper_gen_vec(number)
    return output
    

In [ ]:
def gen_data(number):
    """generates dictionary holding randomly generated problem data for stable marriage problem"""
    source = gen_source(number)
    target = gen_target(number)
    preferences = pd.DataFrame(gen_preferences(number), index = source, columns = target)
    
    return {
        'source': source,
        'target': target,
        'preferences': preferences
    }

In [ ]:
data = gen_data(4)
data['preferences']

,t0,t1,t2,t3
s0,4,3,1,2
s1,1,4,2,3
s2,3,1,2,4
s3,2,1,4,3


In [ ]:
def smp(data):
    m = pyo.ConcreteModel()
    
    m.S = pyo.Set(initialize = data['source'], doc = 'sources s')
    m.T = pyo.Set(initialize = data['target'], doc = 'targets t')
    
    m.x = pyo.Var(m.S, m.T, domain =pyo.Binary, doc ='1 iff s is assigned to t')
    
    @m.Param(m.S,m.T)
    def A(m,s,t):
        return data['preferences'].loc[s,t]
    
    return m

In [ ]:
m = smp(data)
m.pprint()

4 Set Declarations
    A_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    S*T :   16 : {('s0', 't0'), ('s0', 't1'), ('s0', 't2'), ('s0', 't3'), ('s1', 't0'), ('s1', 't1'), ('s1', 't2'), ('s1', 't3'), ('s2', 't0'), ('s2', 't1'), ('s2', 't2'), ('s2', 't3'), ('s3', 't0'), ('s3', 't1'), ('s3', 't2'), ('s3', 't3')}
    S : sources s
        Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {'s0', 's1', 's2', 's3'}
    T : targets t
        Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {'t0', 't1', 't2', 't3'}
    x_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    S*T :   16 : {('s0', 't0'), ('s0', 't1'), ('s0', 't2'), ('s0', 't3'), ('s1', 't0'), ('s1', 't1'), ('s1', 't2'), ('s1', 't3'), ('s2', 't0'), ('s2', 't1'),

In [ ]:
a  = np.random.choice(set_size, replace = False, size = (1,set_size)).tolist()
b  = np.random.choice(set_size, replace = False, size = (1,set_size)).tolist()

In [ ]:
a + b

[[2, 5, 3, 0, 4, 1], [0, 5, 4, 2, 1, 3]]

In [ ]:
pd.DataFrame(a+b)

,0,1,2,3,4,5
0,2,5,3,0,4,1
1,0,5,4,2,1,3
